# KPIs for Food Waste, Insecurity, & Overconsumption


In this notebook, we will explore the impact of food insecurity, meat overconsumption and food waste. 

***Food waste*** is food that is not eaten. Food waste occurs throughout the food system, during production, processing, distribution, retail and consumption. Food waste has a substantial environmental impact and it is a major source of greenhouse emissions. Overbuying, overproduction, and spoilage are the main causes of food waste [[8](#ref_8)]. 

***Food insecurity*** [[1](#ref_1)] is the inability to consume an adequate diet quality or sufficient quantity of food. Household food insecurity is often linked with the household's financial ability to access adequate food. The United States Department of Agriculture (USDA) divides food insecurity into the following 2 categories: Low food security and Very low food security[[2](#ref_2)]. In 2018, 9.2% of the world population were defined as severely food insecure. Food insecurity is highest in Sub-Saharan Africa where nearly one-third are defined as severely insecure [[3](#ref_3)].

***Food overconsumption*** refers to a state in which food intake exceeds individual food requirements, providing an excess of nutrients and energy [[4](#ref_4)]. Almost 20% of the food made available to consumers is lost through over-eating or waste [[5](#ref_5)]. The world population consumes around 10% more food than it needs, while almost 9% is thrown away or left to spoil [[5](#ref_5)]. 

***Meat Overconsumption*** is where a person eats more than their recommended daily intake. Meat overconsumption has been linked to numerous negative health impacts, including: heart disease, obesity, stroke, cancers, type 2 diabetes, and a shorter life span [[6](#ref_6)]. We will mainly focus on the environmental impact of meat overconsumption because it has a huge footprint when it comes to CO2 emissions and water security [[7](#ref_7)].

Food waste, food insecurity and meat overconsumption have roots in a concept called the ***Tragedy of the Commons***. The tragedy of the commons [[11](#ref_11)] is a situation in a shared-resource system where individual users act to maximize their self-interest. They end up depleting the shared resource and behaving in a way contrary to the common good. Overfishing is a great example of this where people try to catch as much fish as they can, which may drive the fish population to extinction. In a similar manner, because of the gap in purchasing power among people, people with larger purchasing power have the incentive to overbuy items without considering the impact of their buying patterns on the environment. A lot of food end up wasted because of this. On the other hand, people with lower purchasing power may face food insecurity because they can't afford buying food.

Here's a great video to explain this concept:

<iframe width="560" height="315" src="https://www.youtube.com/embed/lj_gLquca7Q" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

<br>

We will develop three KPIs that speak to the social and environmental issues facing our world when it comes to food waste, food insecurity, and meat overconsumption. This notebook explicitly emphasizes the social aspect of climate change.

| KPI      |Type| Description |
| :---     | :---         | :---      |
| Food Waste Per Capita Nutrition Loss | Social Vulnerability + Environmental| This KPI measures the per capita nutrition loss expressed in terms of the number of people who could have met their nutritional needs if the food was not wasted.       |
| Food Loss Per Food-Insecure Population| Social Vulnerability + Environmental| This KPI measures the average amount of food in kilograms that a food-insecure person could have received if the food was not wasted.|
| Meat Overconsumption | Environmental + Public Health| This KPI measures the magnitude of meat overconsumption. In addition, it can be used to estimate the  environmental impact of exceeding the meat consumption limit recommended by health experts. Furthermore, it can be used to estimate the CO2 emissions and water waste as a result of exceeding the limit.|

<br>

In the next section, we will introduce the Healthy Diet dataset, which is a dataset that explains the ideal food consumption patterns. 

# Table of Contents
* [Section 1: The Healthy Diet Dataset](#sec_1)
* [Section 2: Food Waste Per Capita Nutrition Loss KPI](#sec_2)
* [Section 3: Food Loss Per Food Insecure KPI](#sec_3)
* [Section 4: Meat Overconsumption KPI](#sec_4)
* [Summary & Conclusion](#sec_conclusion)
* [References](#sec_references)

In [ ]:
from mpl_toolkits.basemap import Basemap
from shapely.geometry import Polygon
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly_express as px
import geopandas as gpd
import seaborn as sns
import folium

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import json
import csv
import re
import os


from plotly.offline import init_notebook_mode;
init_notebook_mode(connected=True)

print(os.getcwd())

In [ ]:
# Some Variables
pie_colors = ["#083069", "#1F6EB1", "#6BAED6", "E7F1FA"]
responses_labels = ['Provided Information', 'No Information']

fwaste_cities_shorter_names = {'eThekwini Municipality': 'eThekwini',  'Comune di Milano': 'Milano',  'Mexico City': 'Mexico City', 'Ajuntament de Barcelona': 'Barcelona', 
                               'City of Sydney': 'Sydney',  'Gemeente Rotterdam': 'Rotterdam', 'City of Athens': 'Athens',  'City of Phoenix': 'Phoenix', 
                               'Dhaka City': 'Dhaka', 'City of Stockholm': 'Stockholm',  'City of Yokohama': 'Yokohama',  'Moscow Government': 'Moscow', 'District of Columbia': 'Washington, DC',
                               'City of Toronto': 'Toronto',  'City of Seattle': 'Seattle', 'Greater London Authority': 'London',  'Auckland Council': 'Auckland',
                               'City of Cape Town': 'Cape Town',  'City of Boston': 'Boston',  'Corporation of Chennai': 'Chennai', 'City of Paris': 'Paris', 
                               'City of Dar es Salaam': 'Dar es Salaam', 'Distrito Metropolitano de Quito': 'Quito',  'Prefeitura do Rio de Janeiro': 'Rio de Janeiro',
                               'The Executive Council, Govt of Dubai': 'Dubai',  'City of San Francisco': 'San Francisco', 'Seoul Metropolitan Government': 'Seoul', 
                               'City of Buenos Aires': 'Buenos Aires', 'Government of Hong Kong Special Administrative Region': 'Hong Kong',
                               'Tokyo Metropolitan Government': 'Tokyo'}
meat_cities_shorter_names = {'City of Delhi': 'Delhi', 'Dhaka City': 'Dhaka', 'Malmö Stad': 'Malmö Stad', 'Municipalidad Distrital de Jesús María': 'Jesús María', 
                             'City of Denizli': 'Denizli','Municipalidad de Comas': 'Comas', 'Corporation of Chennai': 'Chennai', 'Dijon métropole': 'Dijon métropole', 
                             'Göteborgs Stad': 'Göteborgs Stad', 'City of Dar es Salaam': 'Dar es Salaam', 'Municipality of Trelleborg': 'Trelleborg', 
                             'Riga City': 'Riga', 'Gemeente Rotterdam': 'Gemeente Rotterdam', 'Sofia Municipality': 'Sofia', 'Greater Amman Municipality': 'Amman', 
                             'Prefeitura de São Leopoldo': 'São Leopoldo', 'City of Paris': 'Paris', 'Tokyo Metropolitan Government': 'Tokyo', 'Município de Viseu': 'Viseu', 
                             'Greater London Authority': 'London', 'Prefeitura do Rio de Janeiro': 'Rio de Janeiro', 'Seoul Metropolitan Government': 'Seoul', 
                             'The Executive Council, Govt of Dubai': 'Dubai', 'Ayuntamiento de Vitoria-Gasteiz': 'Vitoria-Gasteiz',
                             'Wandsworth Council': 'Wandsworth', 'Richmond Council': 'Richmond', 'Distrito Metropolitano de Quito': 'Quito', 
                             'Stadt Zürich': 'Zürich', 'Municipality of Arendal': 'Arendal', 'Trondheim Municipality': 'Trondheim', 'Egedal Municipality': 'Egedal', 
                             'Gladsaxe Kommune': 'Gladsaxe Kommune', 'City of Hvidovre': 'Hvidovre', 'Hoeje-Taastrup Kommune': 'Hoeje-Taastrup Kommune',
                             'City of Vancouver': 'Vancouver', 'City of Helsinki': 'Helsinki', 'City of Berlin': 'Berlin', 'eThekwini Municipality': 'eThekwini', 
                             'District of Columbia': 'Washington, DC', 'Birmingham City Council': 'Birmingham', 'City of San Leandro, CA': 'San Leandro, CA', 
                             'Region Metropolitana de Guadalajara': 'Guadalajara', 'Umeå municipality': 'Umeå', 'Auckland Council': 'Auckland', 
                             'Bærum Kommune': 'Bærum Kommune', 'City of Turku': 'Turku', 'City of Athens': 'Athens', 'City of Lahti': 'Lahti', 'Comune di Milano': 'Milano',
                             'Comune di Torino': 'Torino', 'City of Espoo': 'Espoo', 'Metropolitan Municipality of Lima': 'Lima', 'Wolverhampton City Council': 'Wolverhampton', 
                             'City of Stockholm': 'Stockholm', 'City of Buenos Aires': 'Buenos Aires', 'BCP Council': 'BCP', 'Tel Aviv-Yafo Municipality': 'Tel Aviv', 
                             'Moscow Government': 'Moscow', 'Municipality of Campinas': 'Campinas', 'Town of Vail, CO': 'Vail, CO', 'City of Baltimore': 'Baltimore', 
                             'City of Houston': 'Houston', 'City of Medford': 'Medford', 'City of Adelaide': 'Adelaide', 'Município de Braga': 'Braga', 
                             'Município de Évora': 'Évora', 'Município de Águeda': 'Águeda', 'City of Boston': 'Boston', 'City of Boulder': 'Boulder', 
                             'Junta Intermunicipal de Medio Ambiente Region Valles (JIMAV)': 'JIMAV',}

In [ ]:
# Helper Functions
def get_city_answers(df, question_name, column_name, new_column_name, include_city, convert_to_numeric, row_name=None, dropna=True, dropNotApplicable=True):
    """
    Gets the answers for a question in the city dataframe.
    
    Parameters
    ----------
        df (dataframe): The city dataframe
        question_name (string): the question to query.
        column_name (string): the column to query.
        new_column_name (string): rename the column name to user friendly name.
        include_city (boolean): if true the organization name column will be included in the returned dataframe.
        convert_to_numeric (boolean): if true, the column will convert to numeric data type.
        row_name(string): Optional, if not None, filter by row.
        dropna(bool): Optional, if true, it will drop invalid rows
        dropNotApplicable (bool): Optional, if true, it will drop Unanswered rows
        
    Returns
    -------
    result_df: The result dataframe for the input question and column
    """
    
    selected_columns = ['Account Number', 'Response Answer']
    renamed_columns = {'Response Answer': new_column_name} 
    if include_city:
        selected_columns = ['Organization'] + selected_columns
        renamed_columns['Organization'] = 'City'


    result_df = df[df['Question Name'] == question_name]
    result_df = result_df[result_df['Column Name'] == column_name]
    if row_name:
        result_df = result_df[result_df['Row Name'] == row_name]
    result_df = result_df[selected_columns].rename(columns=renamed_columns)
        
    # Get information about the amount of response to this question
    total_number_of_answers = len(result_df)
    number_of_invalid_answers = sum(result_df[new_column_name]=='Question not applicable') + sum(result_df[new_column_name].isna()) + sum(result_df[new_column_name] == '0')
    number_of_valid_answers = total_number_of_answers-number_of_invalid_answers
    print ('The number of cities that responded to this questions is', number_of_valid_answers)
    print ('The number that did not respond is', number_of_invalid_answers, 'out of', total_number_of_answers)
    
    # # Drop unanswered responses and invalid answers
    if dropNotApplicable:
        result_df = result_df[result_df[new_column_name]!='Question not applicable']
        result_df = result_df[result_df[new_column_name]!='0']
    if dropna:
        result_df = result_df.dropna()
    
    if convert_to_numeric: 
        result_df[new_column_name] = pd.to_numeric(result_df[new_column_name])

    return result_df, [number_of_valid_answers, number_of_invalid_answers]

def draw_bar_chart(df, x, y, chart_title, x_title, y_title, texttemplate, chart_height, line_x, line_text):
    """
    Draws Bar Chart with Line.
    
    Parameters
    ----------
        df (dataframe): The city dataframe
        x (string): column name for x axis
        y (string): column name for y axis
        chart_title(string): The chart title
        x_title(string): The X axis title
        y_title(string): The Y axis title
        texttemplate(string): The text template for formating numbers
        chart_height(int): The hight of the chart
        line_x (float, optional): if provided a line will be drawn at location x
        line_text (string, optional): line annotation
        
    Returns
    -------
        void
    """

    
    bar_chart = go.Bar(x=df[x], y=df[y], orientation='h', text=df[x], textposition='auto', texttemplate=texttemplate)
    if line_x:
        line = go.Scatter(x=[line_x, line_x], y=[0,.99], mode="lines", marker=dict(color="red"), line=dict(dash='dash'))

    fig = make_subplots(specs=[[{"secondary_y": True}]], print_grid=False)
    fig.add_trace(bar_chart, 1, 1, secondary_y=False)
    if line_x:
        fig.add_trace(line, 1, 1, secondary_y=True)
        fig.add_annotation(x=line_x, y=len(df), text=line_text, showarrow=True, arrowhead=1, bgcolor='red', font=dict(size=16, color="#ffffff"))
    fig.update_layout(title=chart_title, height=chart_height, xaxis=dict(title=x_title), yaxis = dict(title=y_title), yaxis2= dict(fixedrange= True, range= [0, 1], visible= False), showlegend=False)
    fig.show()
    
def draw_pie_chart(labels, values, chart_title, textinfo, colors = None):
    """
    Draws Pie Chart.
    
    Parameters
    ----------
        labels (list): The labels
        values (float): The values
        chart_title(string): The chart title
        textinfo(string): what data to show
        colors(list of strings): the list of colors
        
    Returns
    -------
        void
    """
    
    marker=dict(colors=colors)
    if colors:
        marker['colors'] = colors
    fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo=textinfo, marker=marker)],
                    layout = dict(title=chart_title))
    fig.show()

In [ ]:
# Read all data.
dfs = {}
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        df_path = os.path.join(dirname, filename)
        df_name = filename[:-4]
        if filename.split('.')[-1] == 'csv':
            dfs[df_name] = pd.read_csv(df_path)
        else:
            print('Ignoring Non-csv file: ', filename)

print ('Loaded the following files: ', sorted(list(dfs.keys())))

In [ ]:
# assign dfs to variables to make it easier to work with.
cc_2018_df = dfs['2018_Full_Climate_Change_Dataset']
cc_2019_df = dfs['2019_Full_Climate_Change_Dataset']
cc_2020_df = dfs['2020_Full_Climate_Change_Dataset']

ccd_2018_df = dfs['2018_Corporates_Disclosing_to_CDP_Climate_Change']
ccd_2019_df = dfs['2019_Corporates_Disclosing_to_CDP_Climate_Change']
ccd_2020_df = dfs['2020_Corporates_Disclosing_to_CDP_Climate_Change']

ws_2018_df = dfs['2018_Full_Water_Security_Dataset']
ws_2019_df = dfs['2019_Full_Water_Security_Dataset']
ws_2020_df = dfs['2020_Full_Water_Security_Dataset']

wsd_2018_df = dfs['2018_Corporates_Disclosing_to_CDP_Water_Security']
wsd_2019_df = dfs['2019_Corporates_Disclosing_to_CDP_Water_Security']
wsd_2020_df = dfs['2020_Corporates_Disclosing_to_CDP_Water_Security']

c_2018_df = dfs['2018_Full_Cities_Dataset']
c_2019_df = dfs['2019_Full_Cities_Dataset']
c_2020_df = dfs['2020_Full_Cities_Dataset']

cd_2018_df = dfs['2018_Cities_Disclosing_to_CDP']
cd_2019_df = dfs['2019_Cities_Disclosing_to_CDP']
cd_2020_df = dfs['2020_Cities_Disclosing_to_CDP']

tract_svi_df = dfs['SVI2018_US']# external data - import CDC social vulnerability index data - census tract level
county_svi_df = dfs['SVI2018_US_COUNTY']
cities_locs_df = dfs['uscities']# cities lat,lon locations for US cities
orgs_hq_df = dfs['NA_HQ_public_data']# CDP metadata on organisation HQ cities
recommended_food_serving_df = dfs['recommended_food_serving_df']

# Section 1: The Healthy Diet Dataset
<a id="sec_1"></a>

The Healthy Diet Dataset[[9](#ref_9)] is a small dataset that contains information about food groups along with serving information. The dataset contains 8 food items: vegetables, fruits, grains, dairy, meat, fish, nuts, and fats. For each food group, the following information is provided:
* Serving Per Day
* Serving Per Week
* Serving Size
* Serving Unit (Cups, Oz. , Tbsp)
* Serving Example
* Total Daily Consumption (grams)
* Total Yearly Consumption (kg)
* Number of Servings Per Year

I created this dataset [[9](#ref_9)] and made it public on Kaggle. This dataset is based on an article published at the American Heart Association website [[10](#ref_10)]. The article provides information about the recommended daily servings for each food group. I processed the information and calculated the total yearly consumption of each food group in kilograms. The data is summarized in the following table: 

|Food Group|Example of 1 Serving|Recommended Yearly Consumption Weight (kg)
| :---     | :---     | :---     |
|Vegetables|1/2 cup cut-up vegetables|109.5|
|Fruits|1/2 cup cut-up fruit|87.6|
|Grains|1/2 cup cooked rice, pasta, or cereal|262.8|
|Dairy|1 cup milk or yogurt|262.8|
|Poultry, meat and eggs|3 oz cooked meat or poultry|39.42|
|Fish and other seafood|3 oz cooked fish or seafood|13.14|
|Nuts, seeds, beans and legumes|1/2 cup cooked beans or peas|3.65|
|Fats and oils|1 Tbsp vegetable oil (canola, corn, olive, soybean, safflower)|16.425|
||||
||Total Yearly Food Consumption Weight|795|

<br>

The most important numbers that we will use in this notebook are:
* The total food consumption weight, which is 795 kg per person per year 
* and the total suggested meat and fish weight (39.42 kg for Poultry & red meat + 13.14 kg for fish) = 52.56 kg person per year.

This pie chart shows the suggested amount of food that should be consumed: 

In [ ]:
# The Healthy Diet Dataset is a dataset that provides information about how much food a person eats in 1 year expressed in kilograms.
# I created this dataset and made it public based on an article from: https://www.heart.org/en/healthy-living/healthy-eating/eat-smart/nutrition-basics/suggested-servings-from-each-food-group
# I manually measured how much 1 cup of vegtables and fruit weighs so that all food groups are expressed in kg

# Get the recommended meat consumption
recommended_meat_weight = recommended_food_serving_df[recommended_food_serving_df['food_group'].isin(['Poultry, meat and eggs', 'Fish and other seafood'])]['total_yearly_consumption(kg)'].sum()
print ('Health experts recommend eating', recommended_meat_weight, 'kilograms of meat')

total_weight_recommended_servings = int(round(sum(recommended_food_serving_df['total_yearly_consumption(kg)']), 0))
print ('The total recommended yearly amount of food consumption is around', total_weight_recommended_servings, 'kilograms')

recommended_food_serving_df

In [ ]:
# Draw the Healthy Diet Food Group Breakdown
draw_pie_chart(labels=recommended_food_serving_df['food_group'], 
               values=recommended_food_serving_df['servings_per_year'], 
               chart_title='Healthy Diet', 
               textinfo='label+percent', 
               colors = None)

The top 3 food groups are: grains, vegetables and fruits, which collectively make up 65% of a healthy diet. Health experts recommend eating meat in low quantities. Meat is only 5% of a healthy diet while fish, which is a healthier source of protein, should account for 13% of a healthy diet. 

Because different food groups have different weights, the healthy diet looks different when considering the weight of each food group. As shown in the below chart, grains and dairy account for 66% of healthy diet when looking at food group weights. This is because vegetables and fruits are lighter than grains and dairy. 

In [ ]:
# Draw the Weight of each food group in a Healthy Diet
draw_pie_chart(labels=recommended_food_serving_df['food_group'], 
               values=recommended_food_serving_df['total_yearly_consumption(kg)'], 
               chart_title='The Weights of Different Food Groups that a Person Should Consume in One Year (in Kg)', 
               textinfo='label+value+percent', 
               colors = None)

We are now ready to move on to the next section. In the next section, we will introduce the ***Food Waste Per Capita Nutrition Loss KPI*** after performing some exploratory analysis.

In [ ]:
# Get the population of the city
c_2020_pop_df, c_2020_pop_responses = get_city_answers(df = c_2020_df, 
                                                       question_name = 'Please provide details of your city’s current population. Report the population in the year of your reported inventory, if possible.',
                                                       column_name = 'Current population',
                                                       new_column_name = 'Population',
                                                       include_city = True,
                                                       convert_to_numeric = True)
c_2020_pop_df

In [ ]:
# Get the amount of annual food waste in the city
c_2020_fwaste_df, c_2020_fwaste_responses = get_city_answers(df = c_2020_df, 
                                                             question_name = 'Please report the total annual volume of food waste in tonnes.',
                                                             column_name = 'Total annual volume of food waste in tonnes',
                                                             new_column_name = 'Annual Food Waste (tonnes)',
                                                             include_city = False,
                                                             convert_to_numeric = True)


c_2020_fwaste_df

# Section 2: Food Waste Per Capita Nutrition Loss KPI
<a id="sec_2"></a>

Before we introduce the ***Food Waste Per Capita Nutrition Loss KPI***, we have to explore the information that the cities provided about food waste. As shown in the below pie chart, only 30 cities provided information about the magnitude of food waste. This accounts to around 5% of the cities in the survey.

In [ ]:
# Pie Chart for food waste responses
draw_pie_chart(labels=responses_labels, 
               values=c_2020_fwaste_responses, 
               chart_title='The Number of Cities that Provided Information About Food Waste', 
               textinfo='label+value+percent', 
               colors = pie_colors)

In [ ]:
# Merge Food Waste with Population
c_2020_pop_fwaste_df = pd.merge(c_2020_pop_df, c_2020_fwaste_df, on='Account Number', how='left')

# Compute Food Waste per capita
c_2020_pop_fwaste_df['Annual Food Waste per capita (kg)'] = 1000 * c_2020_pop_fwaste_df['Annual Food Waste (tonnes)'] / c_2020_pop_fwaste_df['Population']
c_2020_pop_fwaste_df = c_2020_pop_fwaste_df.dropna()

c_2020_pop_fwaste_df['City'] = c_2020_pop_fwaste_df['City'].replace(fwaste_cities_shorter_names)

c_2020_pop_fwaste_df = c_2020_pop_fwaste_df.sort_values(by=['Annual Food Waste (tonnes)'], ascending=True)
c_2020_pop_fwaste_df

The below bar chart shows the annual food waste measured in tonnes. The top 5 cities are Tokyo, Hong Kong, Buenos Aires, Seoul, and San Francisco. They reported the annual food waste to be 1.9 million, 1.3 million, 1.2 million, 1 million, and 960 thousand tonnes. These are big cities so it makes sense for them to be at the top of list. 

In [ ]:
# Plot Annual Food Waste
draw_bar_chart(df = c_2020_pop_fwaste_df, 
               x = 'Annual Food Waste (tonnes)', 
               y= 'City', 
               chart_title = 'Cities Annual Food Waste in Tonnes', 
               x_title = 'Annual Food Waste (tonnes)', 
               y_title = 'City', 
               texttemplate = '%{text:.2s}', #'%{text:.2f}', '%{text:.2s}'
               chart_height = 1000, 
               line_x = None,
               line_text = None)

In [ ]:
# Sort By Food Waste Per Capita
c_2020_pop_fwaste_df = c_2020_pop_fwaste_df.sort_values(by=['Annual Food Waste per capita (kg)'], ascending=True)
c_2020_pop_fwaste_df

The next chart shows the annual per capita food waste in kilograms. In other words, it measures how much one person in the city contributes to the food waste issue in the city. As mentioned in [Section 1](#sec_1), the total weight of a healthy diet that is enough to feed one person for one year is 795 kg. The city of San Francisco is on top of the list and it exceeds this limit. In other words, the food waste per one resident in San Francisco can provide enough food for one person for one year. Other cities at the top of the list include Buenos Aires, Boston, Quito, Dubai and Seattle. Tokyo, which was on the top of the list in the previous chart that reports the annual food waste in the city, performs better when looking at the per capita food waste. This is mainly because Tokyo has a large population that is close to 14 million. It should be noted that the chart shows only the cities that provided information about food waste, which as mentioned before is only 5% of the cities in the survey. Therefore, we should expect the total number of cities in the survey that exceed the red dotted line in the chart to be much higher.

In [ ]:
# Cities Annual Per Capita Food Waste in Kilograms
draw_bar_chart(df = c_2020_pop_fwaste_df, 
               x = 'Annual Food Waste per capita (kg)', 
               y= 'City', 
               chart_title = 'Cities Annual Per Capita Food Waste in Kilograms', 
               x_title = 'Annual Per Capita Food Waste (kg)', 
               y_title = 'City', 
               texttemplate = '%{text:.2f}', #'%{text:.2f}', '%{text:.2s}'
               chart_height = 1000, 
               line_x = total_weight_recommended_servings, # None, recommended_meat_weight, 
               line_text = 'Enough Food to Feed <br> One Person For One Year')#'None, Recommened Consumption is '+str(recommended_meat_weight)+' kgs')

In [ ]:
################################################################
# KPI: Food Waste Per Capita Nutrition Loss 

# Description: This KPI measures the per capita nutrition loss expressed in terms of the number of people who could
# have met their nutritional needs if the food was not wasted.

# Type: Social Vulnerability KPI

# Use Case: This KPI can be used to measure the magnitude of the opportunity cost of not being able to avoid 
# food waste. In addition, can be used to raise awareness about the problem of food waste. 

# Example: If the result is 2, it means that the nutrition loss is equal to feeding to 2 people for 1 year.

# Formula: (Annual Food Waste per capita - Annual Food Saved per capita) / Annual Weight of a Healthy Diet
################################################################

food_saved_per_capita = 0
c_2020_pop_fwaste_df['Nutrition Loss Per Capita KPI'] = (c_2020_pop_fwaste_df['Annual Food Waste per capita (kg)'] - food_saved_per_capita) / total_weight_recommended_servings
c_2020_pop_fwaste_df

We are now ready to introduce the ***Food Waste Per Capita Nutrition Loss KPI***. The Food Waste Per Capita Nutrition Loss KPI measures the per capita nutrition loss expressed in terms of the number of people who could have met their nutritional needs if the food was not wasted. For instance, if the result of the KPI is 2, it means that the nutrition loss is equal to feeding 2 people for 1 year. The Food Waste Per Capita Nutrition Loss KPI can be computed as shown below:

$$ Food Waste Per Capita Nutrition Loss = \frac {W_{capita} - S_{capita}} {D} $$

where $W_{capita}$ is the annual food waste per capita, $S_{capita}$ is the annual food saved per capita, and $D$ is the weight of a healthy diet that is sufficient for one person for one year. The cities can minimize this KPI by maximizing annual food saved $S_{capita}$. The Food Waste Per Capita Nutrition Loss KPI is a social vulnerability KPI. It is also an environmental KPI because food waste has a significant carbon footprint. Indeed, 1 kg of food waste produces 2.5 kg of CO2 emissions [[12](#ref_12)]. 

The chart below shows the result of computing this KPI for each city. It should be noted that I set $S_{capita}$ to 0 when creating this chart because we don't have this information. However, in a real life scenario, we can measure the amount of food saved as a result of introducing a new technology or a new initiatives that aims to reduce food waste. There are a lot of initiatives that aim to reduce food waste. A few examples is shown in [[13](#ref_13)].

As shown in the chart below, in San Francisco, the amount of food waste per capita is 1.4, which is equivalent to feeding 1.4 people for one year. 

In [ ]:
#Cities Nutrition Loss Per Capita KPI
draw_bar_chart(df = c_2020_pop_fwaste_df, 
               x = 'Nutrition Loss Per Capita KPI', 
               y= 'City', 
               chart_title = 'Food Waste Per Capita Nutrition Loss <br> (The Number of People whose Nutritional Needs Could Have Been Met If the Food Was Not Wasted)', 
               x_title = 'Number of People', 
               y_title = 'City', 
               texttemplate = '%{text:.2f}', #'%{text:.2f}', '%{text:.2s}'
               chart_height = 1000, 
               line_x = 1, # None, recommended_meat_weight, 
               line_text = 'Enough Food to Feed <br> One Person For One Year')#'None, Recommened Consumption is '+str(recommended_meat_weight)+' kgs')

This is the end of [Section 2](#sec_2). In [Section 3](#sec_3), we will introduce the ***Food Loss Per Food-Insecure Population KPI***.

In [ ]:
# Get the Food Insecurity Information
c_2020_finsec_df, c_2020_finsec_responses = get_city_answers(df = c_2020_df, 
                                                             question_name = 'What percentage of your population is food insecure?',
                                                             column_name = 'Percentage of population that is food insecure', 
                                                             new_column_name = 'Food Insecurity Population Percentage',
                                                             include_city = False,
                                                             convert_to_numeric = True)

c_2020_finsec_df

# Section 3: Food Loss Per Food-Insecure Population KPI
<a id="sec_3"></a>

We will first analyze the information that the cities provided about food insecurity among their residents before introducing the ***Food Loss Per Food-Insecure Population KPI***. The below chart shows the number of cities that provided information about their food-insecure population. Only 34 cities responded to this question, which is about 6% of the cities in the survey.

In [ ]:
# Pie Chart for Food Insecurity responses
draw_pie_chart(labels=responses_labels, 
               values=c_2020_finsec_responses, 
               chart_title='The Number of Cities that Provided Information About Food Insecurity in their Population', 
               textinfo='label+value+percent', 
               colors = pie_colors)

In [ ]:
# Merge Food Waste with Food Insecurity
c_2020_pop_fwaste_finsec_df = pd.merge(c_2020_pop_fwaste_df, c_2020_finsec_df, on='Account Number', how='left')
c_2020_pop_fwaste_finsec_df = c_2020_pop_fwaste_finsec_df.dropna()
c_2020_pop_fwaste_finsec_df = c_2020_pop_fwaste_finsec_df.sort_values(by=['Food Insecurity Population Percentage'], ascending=True)
c_2020_pop_fwaste_finsec_df['Food Insecurity Population Percentage'] = c_2020_pop_fwaste_finsec_df['Food Insecurity Population Percentage']/100
c_2020_pop_fwaste_finsec_df

As shown in the bar chart below, the top 5 cities when it comes to the percentage of food-insecure people in the population are eThekwini (70%), Phoenix (43%), London (38%), Cape Town (31%), and San Francisco (27%). The cities with the lowest percentage of food-insecure people include: Paris (6%), Buenos Aires (5%), Athens (4%), Seoul (3%), Dubai (2%), and Auckland (2%). 

In [ ]:
# The Percentage of the Food-Insecure Population
draw_bar_chart(df = c_2020_pop_fwaste_finsec_df, 
               x = 'Food Insecurity Population Percentage', 
               y= 'City', 
               chart_title = 'The Percentage of the Food-Insecure Population', 
               x_title = 'Food Insecurity Population Percentage', 
               y_title = 'City', 
               texttemplate = '%{text:%}', #'%{text:.2f}', '%{text:.2s}', '%{text:%}'
               chart_height = 1000, 
               line_x = None,
               line_text = None)

In [ ]:
################################################################
# KPI: Food Loss Per Food Insecure

# Description: This KPI measures the average amount of food in kilograms that a food-insecure person could have 
# received if the food was not wasted.

# Type: Social Vulnerability KPI

# Use Cases: This KPI can be used to estimate the impact of food waste in terms of the food-insecure 
# population for programs that try to donate food to vulnerable people before it expires.

# Example: If the result is 10, it means that one person from the food-insecure population could have potentially 
# received 10 kilograms of food.

# Formula: 1000 * (Annual Food Waste - Annual Food Saved) / (Percentage of Food-Insecure Population * Population)
################################################################

food_saved = 0
c_2020_pop_fwaste_finsec_df['Food Insecure Population'] = c_2020_pop_fwaste_finsec_df['Food Insecurity Population Percentage'] * c_2020_pop_fwaste_finsec_df['Population']
c_2020_pop_fwaste_finsec_df['Food Loss per Food-Insecure Person KPI'] = 1000 * (c_2020_pop_fwaste_finsec_df['Annual Food Waste (tonnes)'] - food_saved) / c_2020_pop_fwaste_finsec_df['Food Insecure Population']
c_2020_pop_fwaste_finsec_df = c_2020_pop_fwaste_finsec_df.sort_values(by=['Food Loss per Food-Insecure Person KPI'], ascending=True)

c_2020_pop_fwaste_finsec_df

The ***Food Loss Per Food-Insecure Population KPI*** is a great KPI for programs and initiatives that deliver food to food banks in order to serve it to the food-insecure population before it goes to waste. It measures the average amount of food in kilograms that a food-insecure person could have potentially received if the food was not wasted. For instance, if the result is 10 kg, it means that one person from the food-insecure population could have potentially received 10 kilograms of food. Here is the equation:

$$ Food Loss Per Food Insecure Population KPI = 1000 * \frac {W_{city} - S_{city}} {P_{foodinsecure} * P_{city}} $$

where $W_{city}$ is the city-wide annual food waste, $S_{capita}$ is the city-wide annual food saved,  $P_{foodinsecure}$ is the percentage of food-insecure population, and $P_{city}$ is the population of the city. 

This KPI can be minimized by maximizing the city-wide food that is saved ($S_{city}$). Because the denominator contains the food-insecure population, cities with low food-insecure population will be penalized more. This makes this KPI ideal for cities with small food-insecure populations because it will incentivize them to work even harder to close the gap between food waste and food insecurity. For example, the city of Seoul performs relatively well on the Food Waste Per Capita Nutrition Loss KPI which was introduced in [Section 2](#sec_2). However, as shown in the bar chart below, the city of Seoul is performing worse on the Food Loss Per Food-Insecure Population KPI because it has a relatively small food-insecure population. At the same time, this KPI is not suitable for cities with large food-insecure populations. For instance, the city of eThekwini is doing the best as shown in the chart below and that's because it has a large food-insecure population, which is 70% of its population. Generally speaking, the Food Waste Per Capita Nutrition Loss KPI introduced in [Section 2](#sec_2) should be used by any city regardless of its food-insecure population. If the city has a small food-insecure population, the Food Loss Per Food-Insecure Population KPI is a great incentivizer.

In [ ]:
# The Percentage of the Food-Insecure Population
draw_bar_chart(df = c_2020_pop_fwaste_finsec_df, 
               x = 'Food Loss per Food-Insecure Person KPI', 
               y= 'City', 
               chart_title = 'Food Loss Per Food-Insecure KPI <br>(The Average Amount of Food That Could Have Been Given to 1 Food-Insecure Person )', 
               x_title = 'Food Loss per Food-Insecure Person (kg)', 
               y_title = 'City', 
               texttemplate = '%{text:.2f}', #'%{text:.2f}', '%{text:.2s}', '%{text:%}'
               chart_height = 1000, 
               line_x = None, # None, recommended_meat_weight, 
               line_text = None)#'None, Recommened Consumption is '+str(recommended_meat_weight)+' kgs')

In the next section, we will introduce the last KPI, which can be used to estimate the CO2 emissions and water resulting from meat overconsumption.

In [ ]:
# Get Information About Meat Consumption
c_2020_meat_df, c_2020_meat_responses = get_city_answers(df = c_2020_df, 
                                                         question_name = 'What is the per capita meat and dairy consumption (kg/yr) in your city?',
                                                         column_name = 'Amount', 
                                                         row_name = 'Meat consumption per capita (kg/year)',
                                                         new_column_name = 'Meat Consumption per Capita',
                                                         include_city = False,
                                                         convert_to_numeric = True)

c_2020_meat_df

# Section 4: Meat Overconsumption KPI
<a id="sec_4"></a>

Before we introduce the Meat Overconsumption KPI, let's see the number of cities that provided information about meat overconsumption. As shown in the chart below, 74 cities provided information about the meat consumption per capita.

In [ ]:
# Pie Chart for Meat Consumption responses
draw_pie_chart(labels=responses_labels, 
               values=c_2020_meat_responses, 
               chart_title='The Number of Cities that Provided Information About Meat Consumption Per Capita', 
               textinfo='label+value+percent', 
               colors = pie_colors)

In [ ]:
# Merge the meat and population information
c_2020_pop_meat_df = pd.merge(c_2020_pop_df, c_2020_meat_df, on='Account Number', how='left')
c_2020_pop_meat_df = c_2020_pop_meat_df.dropna()

# Some Cities privided Consumption for the whole population, we will drop these 4 rows
c_2020_pop_meat_df = c_2020_pop_meat_df[c_2020_pop_meat_df['Meat Consumption per Capita']<1000]

# Replace long city names
c_2020_pop_meat_df['City'] = c_2020_pop_meat_df['City'].replace(meat_cities_shorter_names)

c_2020_pop_meat_df = c_2020_pop_meat_df.sort_values(by=['Meat Consumption per Capita'], ascending=True)
c_2020_pop_meat_df

Using the Healthy Diet dataset which we introduced in [Section 1](#sec_1), we know that a well-balanced diet has a limit on the amount of meat that should be consumed in one year. Meat is an excellent source of protein, iron, zinc, selenium and vitamin B [[14](#ref_14)]. However, a person should not eat more than 52.56 kg of meat per year. That's because meat has been linked to numerous negative health impacts, including: heart disease, cancer, diabetes, obesity and stroke [[6](#ref_6)]. Furthermore, meat production has a devastating impact on our environment [[7](#ref_7)]. The next pie chart shows that of the cities that provided information about meat consumption, 52% are exceeding the meat consumption per capita limit. 

In [ ]:
# The Number of Cities that exceeds the Recommended Meat Consumption per Capita
# The recommended meat consumption is 52.56 kgs per year. 

meat_above_limit = sum(c_2020_pop_meat_df['Meat Consumption per Capita'] > recommended_meat_weight)
meat_under_limit = sum(c_2020_pop_meat_df['Meat Consumption per Capita'] <= recommended_meat_weight)

draw_pie_chart(labels=['Above Limit', 'Under Limit'], 
               values=[meat_above_limit, meat_under_limit], 
               chart_title='The Percentage of Cities that Exceeded the Recommended Meat Consumption', 
               textinfo='label+percent', 
               colors = pie_colors)

The bar chart below shows the yearly meat consumption per capita in kilograms along with the recommended consumption (shown as a red line). The consumption of meat in Jimav is 178 kg, which is 3.4 times the recommended limit. In Boston, meat consumption is twice the recommended limit. This has a devastating impact on our environment.

In [ ]:
# Yearly Meat Consumption per Capita (kg)
draw_bar_chart(df = c_2020_pop_meat_df, 
               x = 'Meat Consumption per Capita', 
               y= 'City', 
               chart_title = 'Yearly Meat Consumption per Capita in Kilograms', 
               x_title = 'Yearly Meat Consumption per Capita (kg)', 
               y_title = 'City', 
               texttemplate = '%{text:.2f}', 
               chart_height = 1500, 
               line_x = recommended_meat_weight, 
               line_text = 'Recommened Consumption is '+str(recommended_meat_weight)+' kgs')

In [ ]:
################################################################
# KPI: Meat Overconsumption

# Description: This KPI measures the magnitude of meat overconception. In addition, it can be used to estimate the 
# environmental impact of exceeding the meat consumption limit recommended by health experts. Furthermore, it can
# be used to estimate CO2 and Water waste as a result of exceeding the limit.

# Type: Environmental KPI + General Health KPI

# Use Cases: This KPI can be used to quantify the amount of overconsumed meat in the city. In addition to the 
# environmental issues that this KPI speaks to, it can also be used to raise awarance about the potential
# health issues associated with meat overconsumption such as heart disease, diabetes and cancer.

# Example: If the result is 100, it means that the population of the city collectively overconsumed meat equal 
# to 100 tonnes of meat. 

# Formula: (Annual Meat Consumption per Capita - Annual Recommended Meat Consumption) * Population / 1000
################################################################

c_2020_pop_meat_exceeded_df = c_2020_pop_meat_df[c_2020_pop_meat_df['Meat Consumption per Capita'] > recommended_meat_weight].copy()

c_2020_pop_meat_exceeded_df['Meat Overconsumption KPI (kg)'] = ((c_2020_pop_meat_exceeded_df['Meat Consumption per Capita'] - recommended_meat_weight) * c_2020_pop_meat_exceeded_df['Population'])
c_2020_pop_meat_exceeded_df['Meat Overconsumption KPI'] = ((c_2020_pop_meat_exceeded_df['Meat Consumption per Capita'] - recommended_meat_weight) * c_2020_pop_meat_exceeded_df['Population'])/1000
c_2020_pop_meat_exceeded_df = c_2020_pop_meat_exceeded_df.sort_values(by=['Meat Overconsumption KPI'], ascending=True)

c_2020_pop_meat_exceeded_df


The ***Meat Overconsumption KPI*** quantifies the amount of meat overconsumption. It can be used to estimate the environmental impact of exceeding the meat consumption limit recommended by health experts. Furthermore, it can be used to estimate CO2 and water waste as a result of exceeding the limit. Here's the equation:

$$ Meat Overconsumption KPI = \frac{(M_{capita} - M_{recommended}) *  P_{city}} {1000} $$

where $M_{capita}$ is the amount of the meat consumed per capita , $M_{recommended}$ is the recommended meat consumption, and $P_{city}$ is the population of the city. We divide by 1000 to convert from kg to tonnes. 

This KPI computes the city-wide amount of meat consumed that is above the limit. For example, if the result is 100, it means that the population of the city collectively overconsumed meat equal to 100 tonnes of meat. The bar chart below indicates that Moscow has the highest meat overconsumption among the cities that provided information about meat consumption per capita. The amount of meat overconsumption in Moscow is 550735 tonnes of meat. Lima and Housten come in second and third place with 265022 and 115153 tonnes of meat.

In [ ]:
# Draw Bar Chart Meat Overconsumption in Tonnes KPI
draw_bar_chart(df = c_2020_pop_meat_exceeded_df, 
               x = 'Meat Overconsumption KPI', 
               y= 'City', 
               chart_title = 'Meat Overconsumption in Tonnes KPI<br>(The Amount of Meat Overconsumed in the City)', 
               x_title = 'Meat Overconsumption (Tonnes)', 
               y_title = 'City', 
               texttemplate = '%{text:.2f}', 
               chart_height = 1000, 
               line_x = None, 
               line_text = None)

Meat has the highest carbon footprint among other food groups. The table below shows the amount of CO2 emissions per 1 kg of meat. The information in this table was taken from [[15](#ref_15)]. 

| Meat Type| CO2 Emissions (kg) per 1 kg of meat |
| :---     | :---      |
| Lamb| 39.2|
| Beef| 27|
| Pork| 12.1|
| Farmed Salmon| 11.9|
| Turkey | 10.9|
| Chicken| 6.9|
| Canned Tuna| 6.1|
| Average | 18.6 |

<br>

Assuming that a person consumes equal amounts of these meat types, we can estimate the amount of CO2 emissions in tonnes by multiplying the average CO2 emissions for the various meat types by the Meat Overconsumption KPI. The bar chart below shows the amount of CO2 emissions as a result of producing the over consumed meat. The CO2 emissions for meat overconsumption in Moscow is above 10 million tonnes. The CO2 emissions as a result of meat overconsumption in Lima and Houston is 5 million and 2.1 million, respectively.

In [ ]:
# We can compute CO2 Emissions
# According to: https://www.businessinsider.com.au/the-top-10-foods-with-the-biggest-environmental-footprint-2015-9
# This is how much CO2 these meats produce:
    # 1 kg of Lamb: 39.2 kg CO2
    # 1 kg of Beef: 27 kg CO2
    # 1 kg of Pork: 12.1 kg CO2
    # 1 kg of Farmed Salmon: 11.9 kg CO2
    # 1 kg of Turkey: 10.9 kg CO2
    # 1 kg of Chicken: 6.9 kg CO2
    # 1 kg of Canned Tuna: 6.1 kg CO2

# Assuming that the population is eating equal amounts of each meat type:
    # The Average Meat is: 18.6 kg CO2

meat_to_CO2 = 18.6
c_2020_pop_meat_exceeded_df['Meat Overconsumption CO2 Emissions KPI'] = c_2020_pop_meat_exceeded_df['Meat Overconsumption KPI'] * meat_to_CO2

# Draw Bar Chart Meat Overconsumption CO2 Emissions in Tonnes KPI
draw_bar_chart(df = c_2020_pop_meat_exceeded_df, 
               x = 'Meat Overconsumption CO2 Emissions KPI', 
               y= 'City', 
               chart_title = 'Meat Overconsumption CO2 Emissions in Tonnes KPI', 
               x_title = 'Meat Overconsumption CO2 Emissions (Tonnes)', 
               y_title = 'City', 
               texttemplate = '%{text:.2s}', 
               chart_height = 1000, 
               line_x = None, 
               line_text = None)

Meat overconsumption is also a major challenge to water security. The table below shows the amount of water in liters that is needed to produce 1 kg of meat. The information in this table was taken from [[16](#ref_16)]. 

| Meat Type| Water Needed (liters) per 1 kg of meat |
| :---     | :---      |
| Beef| 15,415|
| Lamb| 10,412|
| Pork| 5,988|
| Chicken| 4,325|
| Average| 9,035|

<br>

In a similar manner, we can multiply the average amount of water needed to produce 1 kg of meat with the Meat Overconsumption KPI to get the amount of water resulting from exceeding the recommended meat limit. In order to visualize the magnitude of the water that went into producing the over consumed meat, we can estimate the amount of water that one person consumes per year. According to the U.S. Geological Survey [[17](#ref_17)], one person consumes 36500 liters per year, which is equivalent to 36.5 Cubic meters per person. We can get the city-wide residential consumption by multiplying this number by the population. 

The chart below shows the meat overconsumption water usage along with the estimated residential water usage in cubic meters. Meat overconsumption exceeds the residential water usage for each city. For instance, in Moscow, meat overconsumption is responsible for 5 Billion cubic meters while residential water usage is far less (462 millions of cubic meters).

In [ ]:
# We can compute Water Cost of Meat 

# source: https://www.theguardian.com/news/datablog/2013/jan/10/how-much-water-food-production-waste
# Here's how much water 1 kg of meat needs:
    # 1 kg of Beef = 15,415 liters
    # 1 kg of Sheep Meat = 10,412  liters
    # 1 kg of Pork = 5,988 liters
    # 1 kg of Chicken meat = 4,325  liters

# Assuming that the population is eating equal amounts of each meat type:
    # The average water used is 9035 liters for 1 kg of meat.

# Estimating The Annual Water Consumption by 1 person
# According to: https://www.usgs.gov/special-topic/water-science-school/science/water-qa-how-much-water-do-i-use-home-each-day?qt-science_center_objects=0#qt-science_center_objects
    # The daily use per person is 80-100 liters per day. We will take the upper limit (100 liters per day) .
    # 100 litres * 365 days = 36500 litres per year = 36.5 Cubic Meters

meat_to_water = 9035 # liters
water_usage_by_person = 36.5# Cubic Water
c_2020_pop_meat_exceeded_df['Meat Overconsumption Water Used KPI'] = c_2020_pop_meat_exceeded_df['Meat Overconsumption KPI'] * meat_to_water
c_2020_pop_meat_exceeded_df['Estimated Water Usage (Cubic Meters)'] = c_2020_pop_meat_exceeded_df['Population'] * water_usage_by_person

# Draw Bar Chart Meat Overconsumption CO2 Emissions in Tonnes KPI
# draw_bar_chart(df = c_2020_pop_meat_exceeded_df, 
#                x = 'Meat Overconsumption Water Used KPI', 
#                y= 'City', 
#                chart_title = 'Water Used to Produce the Overconsumed Meat in Cubic Meters KPI', 
#                x_title = 'Water Used to Produce the Overconsumed Meat (Cubic Meters)', 
#                y_title = 'City', 
#                texttemplate = '%{text:.2s}', 
#                chart_height = 1000, 
#                line_x = None, 
#                line_text = None)

# Plot the Bar Chart
df = c_2020_pop_meat_exceeded_df[-15:]# Show 20 rows only for easier visibility
x, y = 'Meat Overconsumption Water Used KPI', 'City'
x2, y2 = 'Estimated Water Usage (Cubic Meters)', 'City'
chart_title = 'Water Used to Produce the Overconsumed Meat in Cubic Meters KPI'
x_title, y_title = 'Water Used (Cubic Meters)', 'City'
texttemplate = '%{text:.4s}'
chart_height = 1000

text = [str(round(i/1000000, 2))+'M' if i<1000000000 else str(round(i/1000000000, 2))+'B' for i in df[x].values ]

bar_chart = go.Bar(x=df[x], y=df[y], orientation='h', text=text, textposition='auto', name = 'Meat Overconsumption<br>Water Usage')
bar2_chart = go.Bar(x=df[x2], y=df[y2], orientation='h', text=df[x2], textposition='auto', name = 'Residential Water <br> Usage', texttemplate=texttemplate)
fig = make_subplots(specs=[[{"secondary_y": True}]], print_grid=False)
fig.add_trace(bar2_chart, 1, 1, secondary_y=False)
fig.add_trace(bar_chart, 1, 1, secondary_y=False)
fig.update_layout(title=chart_title, height=chart_height, xaxis=dict(title=x_title), yaxis = dict(title=y_title), yaxis2= dict(fixedrange= True, visible= False), showlegend=True)
fig.show()

To sum up, in addition to the health risks of exceeding the meat consumption limit, there's a significant environmental cost to meat overconsumption.

# Summary & Conclusion
<a id="sec_conclusion"></a>

In this notebook, we studied the social and environmental cost of food waste, food insecurity, and meat overconsumption. We introduced the Healthy Diet dataset, which includes information about the different food groups that make up a healthy diet. We used it to study food waste and meat overconsumption. As a result, we introduced three KPIs: the Food Waste Per Capita Nutrition Loss KPI, the Food Loss Per Food-Insecure Population KPI, and the Meat Overconsumption KPI. We used the Meat Overconsumption KPI to study the CO2 emissions and water challenges related to Meat Overconsumption. The table below presents a summary of the three KPIs:

| KPI| Type | Description | Equation | Example | 
| :---     | :---      | :---      | :---      | :---      |
| Food Waste Per Capita Nutrition Loss| Social Vulnerability KPI + Environmental | This KPI measures the per capita nutrition loss expressed in terms of the number of people who could have met their nutritional needs if the food was not wasted. It is ideal for any city regardless of the size of the food-insecure population | $$ Food Waste Per Capita Nutrition Loss = \frac {W_{capita} - S_{capita}} {D} $$ | If the result is 2, it means that the nutrition loss is equal to feeding to 2 people for 1 year. |
| Food Loss Per Food-Insecure Population | Social Vulnerability KPI + Environmental | This KPI measures the average amount of food in kilograms that a food-insecure person could have received if the food was not wasted. It is good for cities with small food-insecure populations as it can incentivize the city to work harder on closing the gap between food waste and insecurity. | $$ Food Loss Per Food Insecure Population KPI = 1000 * \frac {W_{city} - S_{city}} {P_{foodinsecure} * P_{city}} $$ | If the result is 10, it means that one person from the food-insecure population could have potentially received 10 kilograms of food. |
| Meat Overconsumption | Environmental KPI | This KPI measures the magnitude of meat overconsumption. In addition, it can be used to estimate the environmental impact of exceeding the meat consumption limit recommended by health experts. Furthermore, it can be used to estimate CO2 and water waste as a result of exceeding the limit. | $$ Meat Overconsumption KPI = \frac{(M_{capita} - M_{recommended}) *  P_{city}} {1000} $$ | If the result is 100, it means that the population of the city collectively overconsumed meat equal to 100 tonnes of meat.  |



<br>
<br>




Thank you very much for taking the time to read this notebook. 

# References
<a id="sec_references"></a>
* [1] <a id="ref_1"></a> Household food insecurity in Canada: Overview - https://www.canada.ca/en/health-canada/services/food-nutrition/food-nutrition-surveillance/health-nutrition-surveys/canadian-community-health-survey-cchs/household-food-insecurity-canada-overview.html
* [2] <a id="ref_2"></a> Food Insecurity - https://www.healthypeople.gov/2020/topics-objectives/topic/social-determinants-health/interventions-resources/food-insecurity
* [3] <a id="ref_3"></a> Hunger and Undernourishment - https://ourworldindata.org/hunger-and-undernourishment
* [4] <a id="ref_4"></a> Eyeing up intake: an Insight on overconsumption and diet - https://www.foodsecurity.ac.uk/blog/eyeing-up-intake-an-insight-on-overconsumption-and-diet/
* [5] <a id="ref_5"></a> Fifth of world's food lost to over-eating and waste, study finds - https://phys.org/news/2017-02-world-food-lost-over-eating.html
* [6] <a id="ref_6"></a> For Your Health - https://endindustrialmeat.org/ten-reasons-to-opt-out/for-our-health/
* [7] <a id="ref_7"></a> Environmental Impact of Meat Consumption - https://cases.open.ubc.ca/environmental-impact-of-meat-consumption/
* [8] <a id="ref_8"></a> The Problem of Food Waste - https://foodprint.org/issues/the-problem-of-food-waste/
* [9] <a id="ref_9"></a> Healthy Diet Breakdown - https://www.kaggle.com/abdulwahabkabani/healthy-diet-breakdown
* [10] <a id="ref_10"></a> Suggested Servings from Each Food Group - https://www.heart.org/en/healthy-living/healthy-eating/eat-smart/nutrition-basics/suggested-servings-from-each-food-group
* [11] <a id="ref_11"></a> Tragedy of the Commons - https://en.wikipedia.org/wiki/Tragedy_of_the_commons
* [12] <a id="ref_12"></a> The carbon footprint - https://toogoodtogo.org/en/movement/knowledge/the-carbon-footprint
* [13] <a id="ref_13"></a> 21 Inspiring Initiatives Working to Reduce Food Waste Around the World - https://foodtank.com/news/2015/01/twenty-one-inspiring-initiatives-working-to-reduce-food-waste-around-the-wo/
* [14] <a id="ref_14"></a> 3 Benefits of Eating Meat - https://www.medicaldaily.com/3-benefits-eating-meat-234798
* [15] <a id="ref_15"></a> The top 10 foods with the biggest environmental footprint - https://www.businessinsider.com.au/the-top-10-foods-with-the-biggest-environmental-footprint-2015-9
* [16] <a id="ref_16"></a> How Much Water Food Production Waste - https://www.theguardian.com/news/datablog/2013/jan/10/how-much-water-food-production-waste
* [17] <a id="ref_17"></a> Water Q&A: How much water do I use at home each day? - https://www.usgs.gov/special-topic/water-science-school/science/water-qa-how-much-water-do-i-use-home-each-day?qt-science_center_objects=0#qt-science_center_objects